# Dataset Acquisition, Cleaning, and Merging

In [ ]:
# Import libraries 

import pandas as pd
import numpy as np
import requests

## College Scorecard Data

In [ ]:
# Define key and college scorecard API url.
# Note an API key is not included - you will have to provide your own key given from the Scorecard website.
key = ""

url = "https://api.data.gov/ed/collegescorecard/v1/schools"

In [ ]:
# Define fields of interest 
fields = (
    "id,school.name,school.city,school.state,"
    "latest.student.size,"
    "latest.cost.tuition.in_state,latest.cost.tuition.out_of_state,"
    "latest.student.demographics.female_share,"
    "latest.student.demographics.race_ethnicity.white,"
    "latest.student.demographics.race_ethnicity.black,"
    "latest.student.demographics.race_ethnicity.hispanic,"
    "latest.student.demographics.race_ethnicity.asian,"
    "latest.student.demographics.race_ethnicity.aian,"
    "latest.student.demographics.race_ethnicity.nhpi,"
    "latest.student.demographics.race_ethnicity.two_or_more,"
    "latest.staff.faculty.full_time,latest.staff.faculty.part_time,"
    "latest.completion.rate_suppressed.overall"
)


In [ ]:
# Define Parameters for requests 
params = {
    "api_key": key,
    "fields": fields,
    "per_page": 100,
    "page": 0,      
}

In [ ]:
# The following code requests data from the API, the max limit is 1000, so be sure to keep within that range 

# Define results list
all_results = []

# Loop through pages 10 max; 100 observations per page
for page in range(0, 10): 
    params["page"] = page
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        break

    data = response.json()
    results = data.get("results", [])
    if not results:
        break
    all_results.extend(results)

print(f"Retrieved {len(all_results)} records")

# Format the retrieved data from the requests 
df = pd.DataFrame(all_results)

# Rename columns for better readability
df.rename(columns={
    "school.name": "School_Name",
    "school.city": "City",
    "school.state": "State",
    "latest.cost.tuition.in_state": "Tuition_In_State",
    "latest.cost.tuition.out_of_state": "Tuition_Out_State",
    "latest.student.size": "Student_Size",
    "latest.student.demographics.female_share": "Female_Share",
    "latest.completion.rate_suppressed.overall": "Grad_Rate",
    "latest.staff.faculty.full_time": "Faculty_Full_Time",
    "latest.staff.faculty.part_time": "Faculty_Part_Time",
    "latest.student.demographics.race_ethnicity.white": "Race_White",
    "latest.student.demographics.race_ethnicity.black": "Race_Black",
    "latest.student.demographics.race_ethnicity.hispanic": "Race_Hispanic",
    "latest.student.demographics.race_ethnicity.asian": "Race_Asian",
    "latest.student.demographics.race_ethnicity.aian": "Race_AIAN",
    "latest.student.demographics.race_ethnicity.nhpi": "Race_NHPI",
    "latest.student.demographics.race_ethnicity.two_or_more": "Race_Two_Or_More",
}, inplace=True)

print(df.head())
print(df.info())

# Save to csv file. 
df.to_csv("data/college_scorecard_subset.csv", index=False)


Retrieved 1000 records
   Student_Size  Tuition_In_State  Tuition_Out_State  Female_Share  \
0        5726.0           10024.0            18634.0      0.564030   
1       12118.0            8832.0            21864.0      0.639091   
2         226.0               NaN                NaN      0.648649   
3        6650.0           11770.0            24662.0      0.476350   
4        3322.0           11248.0            19576.0      0.613419   

   Race_White  Race_Black  Race_Hispanic  Race_Asian  Race_AIAN  Race_NHPI  \
0      0.0180      0.9043         0.0150      0.0017     0.0012     0.0010   
1      0.4832      0.2657         0.0776      0.0915     0.0020     0.0007   
2      0.2788      0.6681         0.0310      0.0000     0.0044     0.0044   
3      0.6940      0.0907         0.0719      0.0423     0.0074     0.0015   
4      0.0223      0.9082         0.0111      0.0018     0.0015     0.0012   

   Race_Two_Or_More  Grad_Rate                          School_Name  \
0            0.0

In [ ]:
# The population data was retreived from an excel file from the website specified in the README.md file.
# The file was converted into a .csv file the cleaned using the processes below. 
popdata = pd.read_csv("data/NST-EST2024-HU-CUMCHG.csv")

,United States,"140,498,736","146,770,711","6,271,975",4.5,(X),(X).1,(X).2,(X).3,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,.Alabama,"2,288,337","2,381,817","93,480",4.1,24,24,19,20,NaN,NaN,NaN,NaN,NaN,NaN
1,.Alaska,"326,199","330,463","4,264",1.3,50,50,51,48,NaN,NaN,NaN,NaN,NaN,NaN
2,.Arizona,"3,081,997","3,299,651","217,654",7.1,14,14,6,9,NaN,NaN,NaN,NaN,NaN,NaN
3,.Arkansas,"1,365,266","1,421,037","55,771",4.1,31,31,31,21,NaN,NaN,NaN,NaN,NaN,NaN
4,.California,"14,392,141","14,877,904","485,763",3.4,1,1,3,28,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Strip the United States column of the . in front of the state name
popdata['United States'] = popdata['United States'].str.replace('.', '', regex=False).str.strip()

In [ ]:
# Rename the columns 
popdata = popdata.rename(columns={'United States': 'States', '140,498,736': 'April_Estimate_2020', '146,770,711': 'July_Estimate_2024'})

In [ ]:
# Drop any columns that were formed during conversion from excel file to .csv file 
popdata = popdata.drop(['6,271,975', '4.5', '(X)','(X).1','(X).2','(X).3','Unnamed: 9', 'Unnamed: 10','Unnamed: 11',
'Unnamed: 12','Unnamed: 13','Unnamed: 14'], axis=1)

In [ ]:
# Reset index 
popdata.reset_index(drop=True)

,States,April_Estimate_2020,July_Estimate_2024
0,Alabama,"2,288,337","2,381,817"
1,Alaska,"326,199","330,463"
2,Arizona,"3,081,997","3,299,651"
3,Arkansas,"1,365,266","1,421,037"
4,California,"14,392,141","14,877,904"
5,Colorado,"2,491,404","2,676,415"
6,Connecticut,"1,530,193","1,554,121"
7,Delaware,"448,735","476,405"
8,District of Columbia,"350,365","368,736"
9,Florida,"9,865,359","10,629,918"


In [ ]:
# Remove the commas from the esitmates cols and change type to float 
popdata[['April_Estimate_2020', 'July_Estimate_2024']] = (
    popdata[['April_Estimate_2020', 'July_Estimate_2024']]
    .apply(lambda x: x.str.replace(',', '', regex=False).astype(float))
)

In [ ]:
#add abbv for states

state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Map the abbrev to the population data 
popdata['State_Abbrev'] = popdata['States'].map(state_abbrev)

In [15]:
#reorder cols
col_order = ['States', 'State_Abbrev', 'April_Estimate_2020', 'July_Estimate_2024']
popdata = popdata[col_order]

In [ ]:
# save popdata dataframe to csv
popdata.to_csv('data/cleaned_popdata.csv', index=False)

In [ ]:
# read in newly made csv files
popdata_clean = pd.read_csv('data/cleaned_popdata.csv')
college_clean = pd.read_csv('data/college_scorecard_subset.csv')

In [ ]:
# Combine the information from each dataframe to create new combined final product 
combined = college_clean.merge(
    popdata_clean[['State_Abbrev', 'July_Estimate_2024']],
    left_on='State', 
    right_on='State_Abbrev',
    how='left'
)
combined.drop(columns='State_Abbrev', inplace=True)

In [ ]:
#define south vs north to add region column 

state_region = {
    # Southern States
    'AL': 'South', 'AR': 'South', 'FL': 'South', 'GA': 'South', 'KY': 'South',
    'LA': 'South', 'MS': 'South', 'NC': 'South', 'OK': 'South', 'SC': 'South',
    'TN': 'South', 'TX': 'South', 'VA': 'South', 'WV': 'South',
    'DE': 'South', 'MD': 'South', 'DC': 'South',

    # Northern States
    'CT': 'North', 'ME': 'North', 'MA': 'North', 'NH': 'North', 'RI': 'North', 'VT': 'North',
    'NJ': 'North', 'NY': 'North', 'PA': 'North', 'OH': 'North', 'MI': 'North', 'IN': 'North',
    'IL': 'North', 'IA': 'North', 'MN': 'North', 'WI': 'North', 'ND': 'North', 'SD': 'North',
    'NE': 'North', 'KS': 'North', 'MO': 'North', 'MT': 'North', 'WY': 'North', 'CO': 'North',
    'ID': 'North', 'WA': 'North', 'OR': 'North'
}

# Map the region to the combined dataframe
combined['Region'] = combined['State'].map(state_region)

In [ ]:
# Save the final product to csv file. This is the file that will be used in the analysis. 
combined.to_csv('data/college_state_pop.csv', index=False)